# 🚢 EPA Titanic — Logistic Regression V2 (GridSearch)
Ajustamos hiperparámetros de **Regresión Logística** (`C`, `penalty`, `solver`) con **GridSearchCV**.
Compararemos contra V1 para intentar mejorar el score en Kaggle.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


# 📥 Celda 2 — Carga de datos
Leemos los archivos `train.csv` y `test.csv` desde el directorio `/kaggle/input/titanic/`.  
- `train.csv` contiene 891 pasajeros con la variable objetivo `Survived`.  
- `test.csv` contiene 418 pasajeros sin `Survived`, que usaremos para generar las predicciones finales.  
Mostramos las dimensiones y una vista previa de las primeras filas para confirmar que todo está correcto.


In [2]:
# 📥 Celda 2 — Carga de datos (train y test)

import pandas as pd  # librería para manejo de datos en tablas (DataFrames)

# Leemos los archivos oficiales provistos por Kaggle en el entorno de la competición
train_df = pd.read_csv("/kaggle/input/titanic/train.csv")  # dataset de entrenamiento (incluye Survived)
test_df  = pd.read_csv("/kaggle/input/titanic/test.csv")   # dataset de test (sin Survived)

# Mostramos dimensiones para validar tamaños esperados
print("Shape train:", train_df.shape)  # debería ser (891, 12)
print("Shape test :", test_df.shape)   # debería ser (418, 11)

# Vista rápida de las primeras filas para confirmar columnas y tipos
train_df.head()  # muestra las 5 primeras filas


Shape train: (891, 12)
Shape test : (418, 11)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# 🔎 Celda 3 — EDA breve (faltantes y balance de Survived)
Revisamos valores faltantes en `train` y `test`, y verificamos el balance de clases en `Survived`.
Esto nos permitirá definir qué imputaciones serán necesarias antes de entrenar el modelo.


In [3]:
import numpy as np  # librería para operaciones numéricas

# --- Faltantes en TRAIN ---
na_train = train_df.isna().sum()                                # cuenta valores NaN por columna
na_train_pct = (na_train / len(train_df)).round(3)              # calcula % de NaN
faltantes_train = pd.DataFrame({"nulos": na_train, "porcentaje": na_train_pct})
print("Faltantes en train (conteo y %):")
print(faltantes_train.sort_values("nulos", ascending=False))    # ordena de mayor a menor

# --- Faltantes en TEST ---
na_test = test_df.isna().sum()
na_test_pct = (na_test / len(test_df)).round(3)
faltantes_test = pd.DataFrame({"nulos": na_test, "porcentaje": na_test_pct})
print("\nFaltantes en test (conteo y %):")
print(faltantes_test.sort_values("nulos", ascending=False))

# --- Distribución de Survived ---
y_counts = train_df["Survived"].value_counts().sort_index()     # conteo de 0 y 1
y_ratio = train_df["Survived"].value_counts(normalize=True).sort_index().round(4)  # proporciones

print("\nDistribución de Survived (conteo):")
print(y_counts)

print("\nDistribución de Survived (proporción):")
print(y_ratio)


Faltantes en train (conteo y %):
             nulos  porcentaje
Cabin          687       0.771
Age            177       0.199
Embarked         2       0.002
PassengerId      0       0.000
Survived         0       0.000
Pclass           0       0.000
Name             0       0.000
Sex              0       0.000
SibSp            0       0.000
Parch            0       0.000
Ticket           0       0.000
Fare             0       0.000

Faltantes en test (conteo y %):
             nulos  porcentaje
Cabin          327       0.782
Age             86       0.206
Fare             1       0.002
PassengerId      0       0.000
Pclass           0       0.000
Name             0       0.000
Sex              0       0.000
SibSp            0       0.000
Parch            0       0.000
Ticket           0       0.000
Embarked         0       0.000

Distribución de Survived (conteo):
Survived
0    549
1    342
Name: count, dtype: int64

Distribución de Survived (proporción):
Survived
0    0.6162
1    0.38

# 🧱 Celda 4 — Definición de variables base y objetivo
Separamos las columnas que usaremos como **features base** (`X` y `X_test`) y el **target** (`y`).  
Además, guardamos `PassengerId` del test para poder generar el archivo de `submission.csv` al final.

In [4]:
# --- Definición de columnas base ---
# Numéricas originales del dataset
base_num_features = ["Age", "SibSp", "Parch", "Fare", "Pclass"]

# Categóricas originales del dataset
base_cat_features = ["Sex", "Embarked"]

# --- Construcción de X (features de entrenamiento) ---
X = train_df[base_num_features + base_cat_features].copy()  # selecciona columnas base de train

# --- Target (variable objetivo) ---
y = train_df["Survived"].astype(int)  # convertimos Survived a entero (0/1)

# --- Construcción de X_test (features de predicción) ---
X_test = test_df[base_num_features + base_cat_features].copy()  # selecciona columnas base de test

# --- Guardamos PassengerId (para submission) ---
test_passenger_id = test_df["PassengerId"].copy()  # IDs del test

# Vista previa de las primeras filas de X
X.head()


,Age,SibSp,Parch,Fare,Pclass,Sex,Embarked
0,22.0,1,0,7.2500,3,male,S
1,38.0,1,0,71.2833,1,female,C
2,26.0,0,0,7.9250,3,female,S
3,35.0,1,0,53.1000,1,female,S
4,35.0,0,0,8.0500,3,male,S


# 🧪 Celda 5 — Ingeniería de atributos (para LogReg V2)
Creamos nuevas variables dentro de una función que luego integraremos al `Pipeline` vía `FunctionTransformer`:

- `FamilySize = SibSp + Parch + 1`  
- `IsAlone = 1 si FamilySize == 1, si no 0`  
- `Title` (extraído desde `Name`, normalizando y agrupando raros en `Other`)  
- `CabinKnown = 1 si Cabin no es NaN, si no 0`  
- `FarePerPerson = Fare / FamilySize`  
- `TicketGroupSize = cantidad de pasajeros que comparten el mismo Ticket`  

> Nota: Para poder extraer `Title` y calcular `TicketGroupSize`/`CabinKnown`, añadimos temporalmente las columnas auxiliares `Name`, `Ticket` y `Cabin` a `X`/`X_test`.  


In [5]:
# Importamos el transformador funcional para enchufar la FE al pipeline
from sklearn.preprocessing import FunctionTransformer  # permite aplicar una función custom a X dentro del pipeline
import pandas as pd                                    # por si lo necesitamos para Series/DF
import numpy as np                                     # utilidades numéricas

# Definimos la función de ingeniería de atributos
def add_features(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()  # trabajamos sobre una copia para no modificar el objeto original

    # --- FamilySize e IsAlone ---
    df["FamilySize"] = df["SibSp"].fillna(0) + df["Parch"].fillna(0) + 1  # suma familiares + el pasajero
    df["IsAlone"] = (df["FamilySize"] == 1).astype(int)                   # 1 si viaja solo, 0 en caso contrario

    # --- Title (extraído de Name) ---
    if "Name" in df.columns:                                              # nos aseguramos de tener la columna
        titles = df["Name"].str.extract(r",\s*([^\.]+)\.")[0]             # título entre coma y punto
        titles = titles.replace({"Mlle": "Miss", "Ms": "Miss", "Mme": "Mrs"})  # normalizamos variantes
        # nos quedamos con los más frecuentes y agrupamos el resto como 'Other'
        common = ["Mr", "Mrs", "Miss", "Master"]
        titles = titles.where(titles.isin(common), "Other")
        df["Title"] = titles.fillna("Other")                               # rellenamos posibles NaN como 'Other'
    else:
        df["Title"] = "Other"                                              # fallback si no está Name (no debería pasar)

    # --- CabinKnown (bandera si Cabin está presente) ---
    if "Cabin" in df.columns:
        df["CabinKnown"] = (~df["Cabin"].isna()).astype(int)               # 1 si Cabin no es NaN, 0 si lo es
    else:
        df["CabinKnown"] = 0                                               # fallback si no está la columna

    # --- FarePerPerson (evitar div. por cero con replace(0,1)) ---
    df["FarePerPerson"] = df["Fare"].fillna(df["Fare"].median()) / df["FamilySize"].replace(0, 1)

    # --- TicketGroupSize (tamaño del grupo por ticket) ---
    if "Ticket" in df.columns:
        counts = df["Ticket"].map(df["Ticket"].value_counts())             # mapea frecuencia de cada ticket
        df["TicketGroupSize"] = counts.fillna(1).astype(int)               # si no hay match, asumimos 1
    else:
        df["TicketGroupSize"] = 1                                          # fallback

    return df  # devolvemos el DF con las nuevas columnas

# Creamos el transformador para integrarlo al Pipeline en celdas posteriores
feature_engineering = FunctionTransformer(add_features, validate=False)  # validate=False para permitir DataFrame

# Añadimos columnas auxiliares (Name/Ticket/Cabin) a X y X_test para que FE funcione correctamente
for col in ["Name", "Ticket", "Cabin"]:
    if col not in X.columns and col in train_df.columns:  # si no está en X pero sí en train_df, la agregamos alineada por índice
        X[col] = train_df.loc[X.index, col]
    if col not in X_test.columns and col in test_df.columns:  # idem para test
        X_test[col] = test_df.loc[X_test.index, col]

print("✅ Ingeniería de atributos lista (FunctionTransformer definido y columnas auxiliares añadidas).")


✅ Ingeniería de atributos lista (FunctionTransformer definido y columnas auxiliares añadidas).


# 🧽 Celda 6 — Preprocesamiento (imputación, escalado y One-Hot)
Definimos un `ColumnTransformer` que:
- Imputa **numéricas** con **mediana** y luego **escala** (beneficia a LogReg).
- Imputa **categóricas** con **moda** y aplica **One-Hot** (incluye `Title` creado en FE).


In [7]:
# Importamos utilidades para construir el preprocesamiento por tipo de columna
from sklearn.compose import ColumnTransformer                 # aplica transformadores por columna
from sklearn.pipeline import Pipeline                         # encadena pasos (FE -> pre -> modelo)
from sklearn.impute import SimpleImputer                      # imputación de valores faltantes
from sklearn.preprocessing import StandardScaler, OneHotEncoder  # escalado y one-hot

# Definimos las columnas que existirán DESPUÉS del paso de Feature Engineering (Celda 5)  # <- comentario
num_features = [                                              # lista de columnas numéricas finales
    "Age", "SibSp", "Parch", "Fare", "Pclass",               # numéricas originales
    "FamilySize", "IsAlone", "FarePerPerson",                # numéricas creadas
    "TicketGroupSize", "CabinKnown"                          # numéricas/bool creadas
]
cat_features = [                                              # lista de columnas categóricas finales
    "Sex", "Embarked", "Title"                               # categóricas (Title viene de FE)
]

# Pipeline numérico: imputar con mediana y escalar a media 0, std 1
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),           # rellena NaN en numéricas con la mediana
    ("scaler", StandardScaler())                             # estandariza numéricas (crítico para LogReg)
])

# Pipeline categórico: imputar con moda y codificar en dummies densos
categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),    # rellena NaN en categóricas con la moda
    ("onehot", OneHotEncoder(handle_unknown="ignore",        # crea columnas dummy, ignora categorías nuevas
                             sparse_output=False))           # salida densa (array numpy)
])

# Componemos el ColumnTransformer aplicando cada pipeline a su grupo de columnas
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_features),          # aplica pipeline numérico a num_features
        ("cat", categorical_transformer, cat_features)       # aplica pipeline categórico a cat_features
    ],
    remainder="drop"                                         # descarta columnas no listadas (p. ej. Name/Ticket/Cabin)
)

# Mensajes de control para verificar la configuración
print("✅ Preprocesamiento definido.")
print("Num features:", num_features)
print("Cat features:", cat_features)


✅ Preprocesamiento definido.
Num features: ['Age', 'SibSp', 'Parch', 'Fare', 'Pclass', 'FamilySize', 'IsAlone', 'FarePerPerson', 'TicketGroupSize', 'CabinKnown']
Cat features: ['Sex', 'Embarked', 'Title']


# 🔎 Celda 7 — GridSearchCV para Regresión Logística
Buscamos la mejor combinación de hiperparámetros para **LogisticRegression**:

- `C` (fuerza de regularización, menor = más regularización)  
- `penalty` (`l1` o `l2`)  
- `solver` (compatibles con cada `penalty`)  

> Notas:
> - `lbfgs` → solo `l2`  
> - `liblinear` → `l1` o `l2` (binario)  
> - `saga` → `l1` o `l2` (escala mejor con muchas features)  


In [10]:
# Importamos el modelo y utilidades de búsqueda
from sklearn.linear_model import LogisticRegression                  # regresión logística
from sklearn.model_selection import StratifiedKFold, GridSearchCV    # CV y Grid Search
from sklearn.pipeline import Pipeline                                # para encadenar pasos
import numpy as np                                                   # operaciones numéricas
import pandas as pd                                                  # para mostrar resultados ordenados

# Definimos el "estimator" base (los hiperparámetros los variará el grid)
logreg_base = LogisticRegression(max_iter=5000)  # iteraciones altas para asegurar convergencia

# Pipeline completo: FE -> Preprocesamiento -> Modelo
pipe_logreg = Pipeline(steps=[
    ("fe", feature_engineering),   # crea features nuevas (Celda 5)
    ("pre", preprocessor),         # imputación + escalado + one-hot (Celda 6)
    ("logreg", logreg_base)        # regresión logística
])

# Definimos una rejilla de hiperparámetros compatible solver/penalty
# Usamos varios valores de C en escala logarítmica (más fino alrededor de 0.1–10)
C_values = np.logspace(-3, 2, 10)   # 0.001 ... 100

param_grid = [
    # lbfgs con l2
    {
        "logreg__solver": ["lbfgs"],
        "logreg__penalty": ["l2"],
        "logreg__C": C_values
    },
    # liblinear con l1 o l2
    {
        "logreg__solver": ["liblinear"],
        "logreg__penalty": ["l1", "l2"],
        "logreg__C": C_values
    },
    # saga con l1 o l2 (robusto con muchas columnas después del one-hot)
    {
        "logreg__solver": ["saga"],
        "logreg__penalty": ["l1", "l2"],
        "logreg__C": C_values
    }
]

# Esquema de validación cruzada estratificada
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Configuramos el GridSearchCV
grid = GridSearchCV(
    estimator=pipe_logreg,         # pipeline completo
    param_grid=param_grid,         # combinaciones de hiperparámetros
    scoring="accuracy",            # métrica oficial
    cv=cv,                         # validación cruzada
    n_jobs=-1,                     # usa todos los cores disponibles
    refit=True,                    # reentrena el mejor modelo en todo el train
    verbose=1                      # verboso para seguimiento
)

# Ejecutamos la búsqueda
grid.fit(X, y)

# Reportamos el mejor resultado y los mejores hiperparámetros
print("🔝 Mejor accuracy (CV):", grid.best_score_.round(4))
print("🔧 Mejores hiperparámetros:", grid.best_params_)

# Guardamos el mejor pipeline ya reentrenado (listo para predecir)
best_model = grid.best_estimator_

# (Opcional) Mostrar el Top-5 de combinaciones
res = pd.DataFrame(grid.cv_results_).sort_values("mean_test_score", ascending=False)
cols = ["mean_test_score","std_test_score","param_logreg__solver","param_logreg__penalty","param_logreg__C"]
display(res[cols].head(5))


Fitting 5 folds for each of 50 candidates, totalling 250 fits
🔝 Mejor accuracy (CV): 0.8272
🔧 Mejores hiperparámetros: {'logreg__C': 7.742636826811277, 'logreg__penalty': 'l2', 'logreg__solver': 'lbfgs'}


,mean_test_score,std_test_score,param_logreg__solver,param_logreg__penalty,param_logreg__C
25,0.827155,0.010978,liblinear,l2,7.742637
45,0.827155,0.010978,saga,l2,7.742637
24,0.827155,0.010978,liblinear,l1,7.742637
22,0.827155,0.010978,liblinear,l1,2.154435
7,0.827155,0.010978,lbfgs,l2,7.742637


# 📤 Celda 8 — Predicción final y creación de `submission.csv`
Usamos el **mejor pipeline del GridSearch** para predecir en `test` y
guardamos el archivo en el formato requerido por Kaggle.


In [11]:
# 1) Usamos el mejor estimador encontrado por GridSearch (ya está ajustado con refit=True)
#    Si cerraste el kernel, re-ejecuta la Celda 7 antes para tener `best_model` disponible.

# 2) Predicciones sobre el set de test
test_preds = best_model.predict(X_test)          # predicciones 0/1
test_preds = test_preds.astype(int)              # nos aseguramos de tener enteros

# 3) Armamos el DataFrame de submission en el formato oficial
submission = pd.DataFrame({
    "PassengerId": test_passenger_id,           # IDs del test
    "Survived": test_preds                      # predicciones binarias
})

# 4) Chequeos de formato (defensivos)
assert submission.shape[0] == 418, "El submission debe tener exactamente 418 filas."
assert list(submission.columns) == ["PassengerId", "Survived"], "Columnas deben ser PassengerId y Survived."

# 5) Guardamos el CSV
submission.to_csv("submission.csv", index=False)

# 6) Confirmación y vista previa
print("✅ Archivo 'submission.csv' creado con", submission.shape[0], "filas.")
print(submission.head())
print("\nConteo predicciones:", submission["Survived"].value_counts().to_dict())


✅ Archivo 'submission.csv' creado con 418 filas.
   PassengerId  Survived
0          892         0
1          893         1
2          894         0
3          895         0
4          896         1

Conteo predicciones: {0: 252, 1: 166}


# 🏆 Celda 9 — Registro del resultado

- **Score Kaggle:** 0.77033  
- **Notebook:** EPA_TITANIC_LOGREG_V2.ipynb  
- **Modelo:** Regresión Logística con GridSearchCV  
- **Mejores hiperparámetros (CV):** C≈7.74, penalty=l2, solver=lbfgs  
- **Notas:**  
  - CV promedio (5 folds): 0.8272  
  - Score en Kaggle fue **ligeramente peor** que V1 (0.77272).  
  - Probable **sobreajuste por baja regularización** (C alto).  
  - Próximo paso: probar modelos basados en árboles (Random Forest, Gradient Boosting) con estas features.  
